In [2]:
import rioxarray as xr
import os
import glob
import pandas as pd
import xarray
import time
import numpy as np
from datetime import datetime

## Extract features of daily mean temperature

In [2]:
nc_folder_features = "./features/daily_mean_temperature/"
nc_features_paths = glob.glob(nc_folder_features+'*.nc')
nc_features_paths

['./features/daily_mean_temperature/tg_ens_mean_0.1deg_reg_2011-2022_v26.0e.nc',
 './features/daily_mean_temperature/tg_ens_mean_0.1deg_reg_1995-2010_v26.0e.nc']

In [ ]:
for path_name in nc_features_paths:
    features = xarray.open_dataset(path_name)
    features = features['tg']
    features.rio.to_raster(path_name.replace(".nc", ".tif"))

## Extract features of daily precipitation sum

In [ ]:
nc_folder_features = "./features/daily_precipitation_sum/"
nc_features_paths = glob.glob(nc_folder_features+'*.nc')
nc_features_paths

In [ ]:
for path_name in nc_features_paths:
    features = xarray.open_dataset(path_name)
    features = features['rr']
    features.rio.to_raster(path_name.replace(".nc", ".tif"))

In [ ]:
prova = xarray.open_dataset('./features/daily_mean_temperature/tg_ens_mean_0.1deg_reg_2011-2022_v26.0e.nc')

In [ ]:
prova = prova['tg']

In [ ]:
prova

In [ ]:
prova.values[0][1:20]

In [ ]:
prova.rio.write_crs("epsg:4326", inplace=True)
prova = prova.rio.set_spatial_dims(x_dim='longitude', y_dim='latitude')
prova.rio.crs
prova.rio.to_raster("./prova.tif")

In [ ]:
raster = xr.open_rasterio('./features/rasters/tg_ens_mean_0.1deg_reg_2011-2022_v26.0e.tif', mask_and_scale = True)

In [ ]:
np.unique(raster.values[0])[1:20]

In [ ]:
import rasterio
# scaling factor : 0,009999999776482582
with rasterio.open('./features/rasters/tg_1995-2010_Emiliani2.tif') as tmp:
    print(tmp.scales)

In [ ]:
raster = xr.open_rasterio('./features/rasters/tg_ens_mean_0.1deg_reg_2011-2022_v26.0e.tif')

In [ ]:
np.unique(raster.values[0])[1:20]

In [ ]:
raster.read(1)

## Crop tif files in the 10 regions

In [ ]:
shape_files_dir = "./bacini_shp/"
shape_files = glob.glob(shape_files_dir+'*.shp')
shape_files

In [ ]:
tif_files_dir = "./features/rasters/"
tif_files = glob.glob(tif_files_dir+'*.tif')
tif_files

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import mapping
import geopandas as gpd

for tif_name in tif_files: 
    
    raster = xr.open_rasterio(tif_name)
    
    for shape_name in shape_files:
        
        crop_extent = gpd.read_file(shape_name)
        crop_extent = crop_extent.to_crs(epsg=4326)
        raster = raster.rio.set_crs('epsg:4326')
        tiff_clipped = raster.rio.clip(crop_extent.geometry.apply(mapping), crop_extent.crs)
        tiff_clipped.rio.to_raster(tif_name.replace("_ens_mean_0.1deg_reg", "").replace("_v26.0e.tif", "") + 
                                   shape_name.replace("./bacini_shp/", "_").replace(".shp", "") + '.tif')

### Create csv files from cropped tif files (mean values) (cyclostationary mean on training set)

In [4]:
regions = ['Adda',
 'Dora',
 'Emiliani1',
 'Piemonte_Sud',
 'Piemonte_Nord',
 'Oglio_Iseo',
 'Ticino',
 'Garda_Mincio',
 'Lambro_Olona',
 'Emiliani2']

tif_files_dir = "./features/rasters/"
csv_files_dir = "./features/csv/"

# ranges : ['1995-01-01', '2010-12-31'], ['2011-01-01', '2022-06-30']
dates = ['1995-01-01', '2022-06-30']
start = '2001-01-05'

days = pd.date_range(start=dates[0], end=dates[1], freq = 'D')

dates_8days = pd.date_range(start=start, end=dates[1], freq = '8D')
years = [date.year for date in dates_8days]
weeks = [date.isocalendar().week for date in dates_8days]

In [ ]:
def feature_tifs_to_csv(feature):
    cropped_tif_files = [[] for i in range(len(dates))]
    for region in regions:
        cropped_tif_files = glob.glob(tif_files_dir + feature + '*' + region + '*.tif')
        cropped_tif_files.sort()

        for i in range(len(cropped_tif_files)):
            raster = xr.open_rasterio(cropped_tif_files[i]).drop_vars(["spatial_ref"])
            dataframe = raster.to_dataset('band').to_dataframe()

            # remove useless null values
            dataframe = dataframe.replace(-9999,np.NaN)
            dataframe = dataframe.dropna()
            if i == 0:
                complete_dataframe = dataframe
            else:
                complete_dataframe = pd.concat([complete_dataframe, dataframe], axis=1)

        # convert dates in readable ones and remove useless range

        complete_dataframe.columns = days.strftime('%Y-%m-%d')
        complete_dataframe = complete_dataframe.loc[:, complete_dataframe.columns >= start]

        # create dataframe with mean values for each 8 days and save it as csv
        means = complete_dataframe.mean(axis=0)
        means_8days = [means[i:i+8].mean() for i in range(0, len(means), 8)]
        
        statistics = pd.DataFrame({'mean': means_8days, 'year': years, 'week': weeks},
                      index = dates_8days)
        
        statistics.to_csv(csv_files_dir + region + "_" + feature + ".csv")

In [ ]:
feature_tifs_to_csv('tg')
feature_tifs_to_csv('rr')

### Create csv files from cropped tif files (with coordinates)

In [ ]:
csv_files_dir = "./features/csv_allvalues/"

In [5]:
cropped_tif_files = glob.glob(tif_files_dir + "tg" + '*' + "Emiliani2" + '*.tif')
cropped_tif_files.sort()
cropped_tif_files

['./features/rasters/tg_1995-2010_Emiliani2.tif',
 './features/rasters/tg_2011-2022_Emiliani2.tif']

In [16]:
#raster = xr.open_rasterio(cropped_tif_files[0]).drop_vars(["spatial_ref"])
dataframe = raster.to_dataset('band').to_dataframe()

In [17]:
dataframe = dataframe.replace(-9999,np.NaN)
dataframe = dataframe.dropna()

In [18]:

dataframe = dataframe * scale_factor
dataframe

1     2     3     4     5     6     7     8     9     \
y         x                                                                
44.249861 10.44986  0.10 -4.06 -5.46 -7.44 -8.37 -7.23 -8.16 -6.47 -4.64   
          10.54986  1.35 -1.97 -4.61 -6.12 -7.70 -6.99 -7.19 -5.49 -3.21   
44.349861 10.14986  4.59 -1.63 -2.97 -7.19 -6.97 -6.46 -6.04 -4.96 -3.87   
          10.24986  3.64 -2.61 -2.64 -7.27 -7.34 -6.48 -6.15 -5.53 -4.75   
          10.34986  3.68 -2.60 -1.66 -5.73 -6.10 -5.80 -5.09 -4.69 -4.40   
...                  ...   ...   ...   ...   ...   ...   ...   ...   ...   
45.149861 9.94986   7.17  4.81  3.47  1.56  0.55 -0.07 -0.03  0.00 -0.35   
          10.04986  6.24  3.46  2.41  0.31 -0.34 -1.09 -0.90 -1.09 -1.38   
45.249861 9.54986   3.01  1.99  0.31 -0.74 -0.88 -0.84 -0.67 -1.06 -1.02   
          9.84986   6.46  4.14  2.50  0.19 -0.62 -1.01 -0.84 -1.03 -1.05   
          9.94986   6.57  3.80  2.35  0.50 -0.60 -0.89 -1.25 -1.02 -1.27   

                    10    ...  5835  5836  5837  5838  5839  5840  5841  5842  \
y         x               ...                                                   
44.249861 10.44986 -0.36  ...  6.13  5.56  3.43  0.47 -4.19 -8.08 -5.25 -2.39   
          10.54986  1.31  ...  7.12  6.71  4.87  2.24 -2.71 -6.28 -4.03 -0.81   
44.349861 10.14986 -1.12  ...  5.62  4.99  3.83  1.35 -2.89 -6.16 -4.31  0.01   
          10.24986 -1.79  ...  2.76  4.74  4.55  1.95 -2.90 -5.46 -2.93  0.24   
          10.34986 -0.46  ...  2.73  5.20  5.76  3.86 -1.20 -4.99 -3.15 -0.79   
...                  ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   
45.149861 9.94986   3.61  ...  2.28  3.48  5.89  7.17  3.78  0.28 -0.83  0.92   
          10.04986  2.63  ...  2.01  3.34  5.70  6.86  3.55  0.10 -1.07  0.63   
45.249861 9.54986   1.35  ...  0.60  2.11  4.66  5.56  2.17 -1.37 -2.51 -0.78   
          9.84986   3.23  ...  2.11  3.68  5.96  7.07  3.52  0.28 -1.15  0.42   
          9.94986   2.82  ...  2.06  3.58  5.90  7.03  3.46  0.39 -1.16  0.55   

                    5843  5844  
y         x                     
44.249861 10.44986 -0.14  0.41  
          10.54986  1.21  1.97  
44.349861 10.14986  0.26  1.70  
          10.24986  0.14  1.01  
          10.34986  0.86  1.26  
...                  ...   ...  
45.149861 9.94986   2.80  2.50  
          10.04986  2.59  2.11  
45.249861 9.54986   1.19  0.65  
          9.84986   2.66  2.02  
          9.94986   2.56  1.83  

[130 rows x 5844 columns]

In [ ]:
def feature_tifs_to_csv_allcoord(feature):
    cropped_tif_files = [[] for i in range(len(dates))]
    for region in regions:
        cropped_tif_files = glob.glob(tif_files_dir + feature + '*' + region + '*.tif')
        cropped_tif_files.sort()
        for i in range(len(cropped_tif_files)):
            raster = xr.open_rasterio(cropped_tif_files[i]).drop_vars(["spatial_ref"])
            dataframe = raster.to_dataset('band').to_dataframe()

            # remove useless null values
            dataframe = dataframe.replace(-9999,np.NaN)
            dataframe = dataframe.dropna()
            
            scale_factor = raster.attrs['scale_factor']
            dataframe = dataframe * scale_factor # fix the scale factor
            
            if i == 0:
                complete_dataframe = dataframe
            else:
                complete_dataframe = pd.concat([complete_dataframe, dataframe], axis=1)

        complete_dataframe.columns = days.strftime('%Y-%m-%d')
        complete_dataframe = complete_dataframe.loc[:, complete_dataframe.columns >= start]

        # create a multi_index with both coordinates and date
        multi_index_dataframe = pd.concat([complete_dataframe] * len(dates_8days), keys=dates_8days, names=['date'])

        # save mean values for groups of 8 days
        for i in range(0, len(complete_dataframe.columns), 8):
            if i == 0:
                cells_means_8days = complete_dataframe.iloc[:,i:i+8].mean(axis = 1).values
            else:
                cells_means_8days = np.concatenate([cells_means_8days, complete_dataframe.iloc[:,i:i+8].mean(axis = 1).values])

        statistics = pd.DataFrame({'mean': cells_means_8days, 'year': np.repeat(years, len(complete_dataframe)), 'week': np.repeat(weeks, len(complete_dataframe))},
              index = multi_index_dataframe.index)

        n = 0.6
        # take only the first 60% of the dataframe and compute the cyclostationary mean for week
        last_training_day = round(len(dates_8days)*n)

        train_df = statistics[statistics.index.get_level_values(0) < dates_8days[last_training_day]]
        # cyclostationary_means_8days
        weekoftheyar_mean = train_df.groupby(['week', 'y', 'x'])['mean'].mean()
        index = statistics.index
        statistics = pd.merge(statistics, weekoftheyar_mean, how='left', on=['week', 'y', 'x'], suffixes=['','_weekoftheyear']).set_index(index)
        statistics['cyclostationary_mean'] = statistics['mean'] - statistics['mean_weekoftheyear']
        statistics.drop("mean_weekoftheyear", axis='columns', inplace = True)

        statistics.to_csv(csv_files_dir + region + "_" + feature + ".csv")

In [ ]:
feature_tifs_to_csv_allcoord('tg')
feature_tifs_to_csv_allcoord('rr')

In [ ]:
region = "Emiliani2"
feature = "tg"